In [1]:
class NewickTree:
    def __init__(self, elements, left_child, right_child, left_branch, right_branch):
        self.elements = elements
        self.l = left_child
        self.r = right_child
        self.left_branch = left_branch
        self.right_branch = right_branch

    def to_string(self):
        if len(self.elements) == 1:
            return str(list(self.elements)[0])
        else:
            a = "(" + self.l.to_string() + ":" + str(self.left_branch) + "," + self.r.to_string() + ":" + str(
                self.right_branch) + ")"
            return a

In [2]:
def find_min(table, sum_of_distances):
    minimum = 1000101010100000
    A = None
    B = None
    n = len(table)

    for (a, b) in table.items():
        for (c, d) in b.items():
            if d == 0:
                continue
            dist = d - (sum_of_distances[a] + sum_of_distances[c] - 2 * d) / (n - 2)
            if dist < minimum:
                A = a
                B = c
                minimum = dist
    return A, B

In [3]:
def nj(table):
    cluster_distances = table
    tree = {}
    for (a, _) in table.items():
        tree[a] = NewickTree(a, None, None, 0.0, 0.0)
    while True:
        sumOfDistances = {}
        for (a, b) in cluster_distances.items():
            sumOfDistances[a] = 0
            for (c, d) in b.items():
                sumOfDistances[a] = sumOfDistances[a] + d

        (a, b) = find_min(cluster_distances, sumOfDistances)
        new_cluster = a.union(b)
        new_cluster_distances = {new_cluster: {new_cluster: 0}}
        n = len(cluster_distances)
        a_branch_length = (cluster_distances[a][b] + (sumOfDistances[a] - sumOfDistances[b]) / (n - 2)) / 2
        b_branch_length = (cluster_distances[a][b] + (sumOfDistances[b] - sumOfDistances[a]) / (n - 2)) / 2
        tree[new_cluster] = NewickTree(new_cluster, tree[a], tree[b], a_branch_length, b_branch_length)

        for (k, v) in cluster_distances[a].items():
            if k == a or k == b:
                continue
            new_cluster_distances[new_cluster][k] = (cluster_distances[a][k] + cluster_distances[b][k] -
                                                     cluster_distances[a][b]) / 2
            if not (k in new_cluster_distances):
                new_cluster_distances[k] = {k: 0.0}
            new_cluster_distances[k][new_cluster] = new_cluster_distances[new_cluster][k]

        for (k1, v1) in cluster_distances.items():
            if k1 == a or k1 == b:
                continue
            for (k2, v2) in v1.items():
                if k2 == a or k2 == b:
                    continue
                new_cluster_distances[k1][k2] = v2
        if len(new_cluster_distances) == 2:
            fst = new_cluster
            for i in new_cluster_distances[new_cluster].keys():
                if i != new_cluster:
                    snd = i
                    break
            new_new_cluster = fst.union(snd)
            tree[new_new_cluster] = NewickTree(new_new_cluster, tree[fst], tree[snd],
                                               new_cluster_distances[fst][snd] / 2,
                                               new_cluster_distances[fst][snd] / 2)
            return tree[new_new_cluster]
        cluster_distances = new_cluster_distances

In [4]:
test1 = {
    frozenset({"A"}): {frozenset({"A"}): 0.0, frozenset({"B"}): 16.0, frozenset({"C"}): 16.0, frozenset({"D"}): 10.0},
    frozenset({"B"}): {frozenset({"A"}): 16.0, frozenset({"B"}): 0.0, frozenset({"C"}): 8.0, frozenset({"D"}): 8.0},
    frozenset({"C"}): {frozenset({"A"}): 16.0, frozenset({"B"}): 8.0, frozenset({"C"}): 0.0, frozenset({"D"}): 4.0},
    frozenset({"D"}): {frozenset({"A"}): 10.0, frozenset({"B"}): 8.0, frozenset({"C"}): 4.0, frozenset({"D"}): 10.0}}

test2 = {
    frozenset({"A"}): {frozenset({"A"}): 0, frozenset({"B"}): 5, frozenset({"C"}): 4, frozenset({"D"}): 7,
                       frozenset({"E"}): 6, frozenset({"F"}): 8},
    frozenset({"B"}): {frozenset({"A"}): 5, frozenset({"B"}): 0, frozenset({"C"}): 7, frozenset({"D"}): 10,
                       frozenset({"E"}): 9, frozenset({"F"}): 11},
    frozenset({"C"}): {frozenset({"A"}): 4, frozenset({"B"}): 7, frozenset({"C"}): 0, frozenset({"D"}): 7,
                       frozenset({"E"}): 6, frozenset({"F"}): 8},
    frozenset({"D"}): {frozenset({"A"}): 7, frozenset({"B"}): 10, frozenset({"C"}): 7, frozenset({"D"}): 0,
                       frozenset({"E"}): 5, frozenset({"F"}): 9},
    frozenset({"E"}): {frozenset({"A"}): 6, frozenset({"B"}): 9, frozenset({"C"}): 6, frozenset({"D"}): 5,
                       frozenset({"E"}): 0, frozenset({"F"}): 8},
    frozenset({"F"}): {frozenset({"A"}): 8, frozenset({"B"}): 11, frozenset({"C"}): 8, frozenset({"D"}): 9,
                       frozenset({"E"}): 8, frozenset({"F"}): 0}
}

In [5]:
def run(test_number):
    if test_number == 1:
        test = test1
    elif test_number == 2:
        test = test2
    else:
        print("No such test")
        return
    print(nj(test).to_string())

In [6]:
run(1)

(((C:1.0,D:3.0):0.5,B:5.5):5.25,A:5.25)


In [7]:
run(2)

(((((A:1.0,B:4.0):1.0,C:2.0):1.5,E:2.5):-0.5,D:3.5):2.75,F:2.75)
